When you download from Google Drive, folder structure and files comes in a bunch of different Zip folders. This is to reform the folder/file structure in NextCloud

In [ ]:
# this script creates the folder structure. note that it does modify folder names and records original vs. changed folder names

import os
import zipfile
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

MAX_FOLDER_NAME_LENGTH = 50  # Maximum length for each folder name segment
MAX_PATH_LENGTH = 260        # Maximum length for the entire path

def select_zip_files():
    """Open file dialog to select multiple ZIP files."""
    zip_files = filedialog.askopenfilenames(
        title="Select ZIP files",
        filetypes=[("ZIP files", "*.zip")]
    )
    return zip_files

def select_output_directory():
    """Open directory dialog to select output folder."""
    output_directory = filedialog.askdirectory(title="Select Output Directory")
    return output_directory

def sanitize_and_truncate_folder_name(folder_name):
    """Sanitize folder name by removing trailing whitespace and truncating if needed."""
    folder_name = folder_name.strip()  # Remove trailing spaces
    if len(folder_name) > MAX_FOLDER_NAME_LENGTH:
        truncated_name = folder_name[:MAX_FOLDER_NAME_LENGTH]
        return truncated_name, folder_name  # Return truncated and original names
    return folder_name, None  # No truncation needed

def create_folder_structure_with_progress(zip_files, output_directory, progress_bar, progress_label):
    """Reconstruct folder structure from ZIP files with truncation for long paths and progress tracking."""
    error_log = []  # Log to capture folder creation issues
    truncation_log = []  # To record original folder names for truncated folders

    total_entries = sum(len(zipfile.ZipFile(zip_file).namelist()) for zip_file in zip_files)
    processed_entries = 0

    for zip_file in zip_files:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            for member in zip_ref.namelist():
                # Get the directory path
                original_directory_path = os.path.join(output_directory, os.path.dirname(member))
                truncated_directory_path = output_directory

                # Process each folder level in the path
                for part in os.path.normpath(original_directory_path).split(os.sep):
                    sanitized_name, original_name = sanitize_and_truncate_folder_name(part)
                    truncated_directory_path = os.path.join(truncated_directory_path, sanitized_name)

                    # Record truncation if it occurred
                    if original_name:
                        truncation_log.append(f"Truncated '{original_name}' to '{sanitized_name}' in path: {truncated_directory_path}")

                # Ensure path length does not exceed Windows limit
                if len(truncated_directory_path) > MAX_PATH_LENGTH:
                    error_log.append(f"Path too long: {truncated_directory_path}")
                    continue

                # Create the directory if it doesn't exist
                try:
                    if not os.path.exists(truncated_directory_path):
                        os.makedirs(truncated_directory_path, exist_ok=True)
                except FileNotFoundError as e:
                    # Log specific error for debugging
                    error_log.append(f"Error creating {truncated_directory_path}: {e}")
                    continue

                # Update progress
                processed_entries += 1
                progress = processed_entries / total_entries * 100
                progress_bar["value"] = progress
                progress_label.config(text=f"Processing folder {processed_entries} of {total_entries}")
                progress_bar.update_idletasks()

    # Write truncation log to a file
    if truncation_log:
        with open(os.path.join(output_directory, "original_folder_names.txt"), "w") as log_file:
            log_file.write("\n".join(truncation_log))

    # Log any errors
    if error_log:
        with open("folder_creation_error_log.txt", "w") as log_file:
            log_file.write("\n".join(error_log))
        messagebox.showwarning("Folder Creation Completed with Errors", 
                               "Folder structure was created, but some errors occurred. Check 'folder_creation_error_log.txt' for details.")
    else:
        messagebox.showinfo("Success", "Folder structure created successfully!")

def main():
    # Set up tkinter window
    root = tk.Tk()
    root.title("Rebuild Folder Structure from ZIP Files")
    root.geometry("400x300")

    # Instruction label
    label = tk.Label(root, text="Select ZIP files and an output directory to recreate folder structure")
    label.pack(pady=10)

    # Progress bar and label
    progress_label = tk.Label(root, text="Progress: 0%")
    progress_label.pack(pady=5)
    progress_bar = ttk.Progressbar(root, orient="horizontal", length=300, mode="determinate")
    progress_bar.pack(pady=5)

    # Start button
    def start_structure_creation():
        zip_files = select_zip_files()
        output_directory = select_output_directory()

        if not zip_files or not output_directory:
            messagebox.showerror("Error", "Please select both ZIP files and an output directory.")
            return

        # Create folder structure only with progress and truncation handling
        create_folder_structure_with_progress(zip_files, output_directory, progress_bar, progress_label)

    # Start button
    start_button = tk.Button(root, text="Start Folder Structure Creation", command=start_structure_creation)
    start_button.pack(pady=20)

    # Run tkinter main loop
    root.mainloop()

if __name__ == "__main__":
    main()


In [4]:
import os
import zipfile
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk

def select_zip_files():
    """Open file dialog to select multiple ZIP files."""
    zip_files = filedialog.askopenfilenames(
        title="Select ZIP files",
        filetypes=[("ZIP files", "*.zip")]
    )
    return zip_files

def select_output_directory():
    """Open directory dialog to select output folder."""
    output_directory = filedialog.askdirectory(title="Select Output Directory")
    return output_directory

def count_files_in_zips(zip_files):
    """Count total number of files in all ZIP files."""
    total_files = 0
    for zip_file in zip_files:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            total_files += len([f for f in zip_ref.namelist() if not f.endswith('/')])
    return total_files

def extract_files_to_existing_structure(zip_files, output_directory, progress_bar, progress_label):
    """Extracts files from multiple ZIPs into an existing folder structure with progress tracking."""
    total_files = count_files_in_zips(zip_files)
    files_processed = 0
    error_log = []  # To store any errors that occur

    for zip_file in zip_files:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            for member in zip_ref.namelist():
                if member.endswith('/'):
                    # Skip directories, as we're only extracting files into an existing structure
                    continue

                # Determine the target path for each file
                target_path = os.path.join(output_directory, member)

                try:
                    # Ensure the parent directory exists
                    target_dir = os.path.dirname(target_path)
                    if os.path.exists(target_dir):
                        # If file doesn't exist in the target location, extract it
                        if not os.path.exists(target_path):
                            with zip_ref.open(member) as source, open(target_path, "wb") as target_file:
                                shutil.copyfileobj(source, target_file)
                    else:
                        # Log if the required directory doesn't exist in the output structure
                        error_log.append(f"Directory missing for {member}, unable to extract to {target_path}")
                        continue

                    # Update progress
                    files_processed += 1
                    progress = files_processed / total_files * 100
                    progress_bar["value"] = progress
                    progress_label.config(text=f"Processed {files_processed} of {total_files} files")
                    progress_bar.update_idletasks()

                except Exception as e:
                    # Log specific errors for later review
                    error_log.append(f"Error extracting {member} to {target_path}: {e}")
                    continue

    # Display completion message and save errors if any
    if error_log:
        with open("file_extraction_error_log.txt", "w") as log_file:
            log_file.write("\n".join(error_log))
        messagebox.showwarning("Extraction Completed with Errors", 
                               f"Extraction completed with some errors. Check 'file_extraction_error_log.txt' for details.")
    else:
        messagebox.showinfo("Success", "Files extracted and placed into the existing folder structure successfully!")

def main():
    # Set up tkinter window
    root = tk.Tk()
    root.title("Fill Existing Folder Structure with Files from ZIPs")
    root.geometry("400x250")

    # Instruction label
    label = tk.Label(root, text="Select ZIP files and an output directory with existing folder structure")
    label.pack(pady=10)

    # Progress bar and label
    progress_label = tk.Label(root, text="Progress: 0 files processed")
    progress_label.pack(pady=5)
    progress_bar = ttk.Progressbar(root, orient="horizontal", length=300, mode="determinate")
    progress_bar.pack(pady=5)

    def start_extraction():
        zip_files = select_zip_files()
        output_directory = select_output_directory()

        if not zip_files or not output_directory:
            messagebox.showerror("Error", "Please select both ZIP files and an output directory.")
            return

        # Extract files to the existing folder structure
        extract_files_to_existing_structure(zip_files, output_directory, progress_bar, progress_label)

    # Start button
    start_button = tk.Button(root, text="Start File Extraction", command=start_extraction)
    start_button.pack(pady=20)

    # Run tkinter main loop
    root.mainloop()

if __name__ == "__main__":
    main()
